1. 安装 LlamaIndex 依赖包

In [6]:
%pip install -r ./requirements.txt

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [13]:
import os

from config.load_key import load_key
load_key()

# 生产环境中请勿将 API Key 输出到日志中，避免泄露
print(f'''你配置的 API Key 是：{os.environ["DASHSCOPE_API_KEY"][:5]+"*"*5}''')

你配置的 API Key 是：sk-9a*****


文件解析

In [1]:
from llama_index.llms.dashscope import DashScope
from llama_index.core.base.llms.types import MessageRole, ChatMessage

# 定义 llm 用于后续和大模型交互
llm = DashScope(model_name="qwen-plus")

messages = [
    ChatMessage(role=MessageRole.SYSTEM, content="你是智能助理贾维斯，分别用中文和英文回答我的问题。"),
    ChatMessage(role=MessageRole.USER, content="你叫什么名字？")
    ]

# notebook 中运行代码块后，会打印最后一个变量，无需使用 print 方法
llm.chat(messages).message.content

'中文：我叫贾维斯，是您的智能助手。\n英文：My name is JARVIS, I am your intelligent assistant.'

In [2]:
# 流式回答
responses = llm.stream_chat(messages)
for response in responses:
    print(response.delta, end="")

中文：我叫贾维斯，是您的智能助手。
英文：My name is JARVIS, I am your intelligent assistant.

In [3]:
import os

from llama_index.readers.dashscope.base import DashScopeParse
from llama_index.readers.dashscope.utils import ResultType
from llama_index.embeddings.dashscope import DashScopeEmbedding,DashScopeTextEmbeddingModels
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex


# 忽略警告级别的日志
# import logging
# logging.basicConfig(level=logging.ERROR)


# 多个独立文件
# file = [
#     # 需要解析的文件，支持pdf,doc,docx
# ]
# parse = DashScopeParse(result_type=ResultType.DASHSCOPE_DOCMIND)
# documents = parse.load_data(file_path=file)

# 使用SimpleDirectoryReader读取文件夹下所有文件
from llama_index.core import SimpleDirectoryReader
parse = DashScopeParse(result_type=ResultType.DASHSCOPE_DOCMIND)
file_extractor = {".pdf": parse, '.doc': parse, '.docx': parse}
# 使用 Data connectors 将不同格式注入并转化为 Llamaindex 支持的 Document
# Document 是 Llamaindex 中容器，Node 是 Llamaindex 中最小单元，是 Document 的分块
documents = SimpleDirectoryReader(
   "./files", file_extractor=file_extractor
).load_data(num_workers=1)
print(documents)
print("完成文件解析和读取")

[Document(id_='aaa697db-1f67-4333-87f5-1b0d1f8dcf42', embedding=None, metadata={'file_path': 'e:\\AI_Python_for_Intermediates\\Chap5\\files\\name.txt', 'file_name': 'name.txt', 'file_type': 'text/plain', 'file_size': 76, 'creation_date': '2024-11-02', 'last_modified_date': '2024-11-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='我的名字是贾维斯\r\n\r\n我的回复方式是中文和英文同时回复', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')]
完成文件解析和读取


2. 创建索引

In [4]:
from llama_index.indices.managed.dashscope import DashScopeCloudIndex

# create a new index
index = VectorStoreIndex.from_documents(
    documents,
    # 指定embedding 模型
    embed_model=DashScopeEmbedding(
        model_name=DashScopeTextEmbeddingModels.TEXT_EMBEDDING_V3
    ))

print("完成向量化和索引构建")

# 更多embedding模型：
# https://help.aliyun.com/zh/model-studio/getting-started/models#3383780daf8hw

完成向量化和索引构建


3. 读取索引

In [5]:
my_index = DashScopeCloudIndex("my_first_index")

4. 创建提问引擎

In [6]:
query_engine = index.as_query_engine(
    # 设置为流式输出
    streaming=True,
    # 设置大模型名称
    llm=DashScope(model_name="qwen-plus")
    )
print("完成查询引擎构建")

# 更多文本生成模型：
# https://help.aliyun.com/zh/model-studio/getting-started/models#9f8890ce29g5u

完成查询引擎构建


5. 生成回复

In [7]:
# 封装了很多方法：
# Retriever：检索器，用于从索引中检索相关文档
# RAG管道：用于处理用户查询并生成响应
#   QueryEngine：查询引擎，用于处理用户查询并生成响应
#   ChatQueryEngine：聊天查询引擎，用于处理用户聊天查询并生成响应
#   Agent：基于用户意图动态决策，不使用预定逻辑

streaming_response = query_engine.query('你的名字是什么？')
print("正在生成回复，请稍等")

正在生成回复，请稍等


6. 显示生成结果

In [8]:
streaming_response.print_response_stream()
print("完成查询")

我的名字是贾维斯。My name is Jarvis.完成查询
